# Preparando el proceso de ingesta de datos.

En este notebook vamos a realizar el proceso de ingesta de datos nuevos para la base de datos y poder así poder tener información continua para realizar un modelado de datos.

En este punto vamos a proceder a trabajar con código que hemos ido desarrollando en los anteriores notebooks.

    - Obtención de datos de la página web, pero aplicado a un determinado tiempo, que será periódico.
    - Ingeniería básica para preparar los .csv necesarios  para la carga de datos en la BBDD.
    - Carga de datos nuevos en la BBDD.

Vamos a intentar reducir el codigo aplicado en relación con los notebooks anteriores. Para ello vamos a recurrir a funciones que generaremos a partir de los codigos anteriores y que estarán en el fichero de [**functions.py**](..\Utils\functions.py)

Es por ello que aquí se verá solo el código sencillo de llamamiento a funciones donde generará primero unos ficheros .csv para después cargar los mismos en la base de datos.

## Descargando los datos para su ingesta

Aquí vamos a desarrollar las funciones necesarias para obtener la información de la página web. Así como las modificaciones que consideramos oportunas para adecuar la información a las tablas creadas en nuestra BBDD.

### Primeramente vamos a convertir en funciones lo que tenemos ahora.

In [ ]:
import os
os.chdir(os.path.split(os.getcwd())[0])

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
import os
import pickle
import re
from datetime import datetime
import spacy
from nltk.stem.snowball import SnowballStemmer
from Utils import functions as f

In [ ]:
'''Variables'''

url_principal="https://www.amantis.net/"                        
url_secundaria=url_principal+'productos-amantis/'


date=str(datetime.today().strftime('%y%m%d'))
folder_ingest=r'.\Data'

ext=r'.csv'
scrape='_scrape'
file_product=r'\productos'
file_user=r'\usuarios'
file_comment=r'\comentarios'
file_price=r'\precios'
file_tag=r'\tags'
file_ingest_product=file_product+scrape
file_ingest_comment=file_comment+scrape

'''Mapeo de meses'''
dm_mapping={
    'enero':1, 
    'febrero':2, 
    'marzo':3, 
    'abril':4, 
    'mayo':5,
    'junio':6, 
    'julio':7,
    'agosto':8, 
    'septiembre':9, 
    'octubre':10, 
    'noviembre':11, 
    'diciembre':12,
} 

nombre_listas=['amenities','anal','BDSM','femenino','masculino','juguetes','lenceria','muebles']

product_ingest=folder_ingest+file_ingest_product+'_'+date+ext
comment_ingest=folder_ingest+file_ingest_comment+'_'+date+ext
product_engineer=folder_ingest+file_product+'_'+date+ext
tag_engineer=folder_ingest+file_tag+'_'+date+ext
price_engineer=folder_ingest+file_price+'_'+date+ext
comment_engineer=folder_ingest+file_comment+'_'+date+ext
user_engineer=folder_ingest+file_user+'_'+date+ext


In [ ]:
def spider_amantis(url=url_secundaria,product_ingest=product_ingest,comment_ingest=comment_ingest):
    """
    Obtiene las URLs de los productos para luego extraer la información de cada producto.

    Input:
    - url (str): URL base para la extracción de productos.

    Return:
    - noduplicated_product (pd.DataFrame): DataFrame de productos sin duplicados.
    - noduplicated_comments (pd.DataFrame): DataFrame de comentarios sin duplicados.
    """
    
    pages= np.arange(1,25)  
    '''Listas a generar con la información de los productos'''
    lista_URLs = []
    name=[]
    regular_prices=[]
    new_price=[]
    info=[]
    id=[]
    comentarios=[]
    '''Generamos 2 diccionarios con los datos importantes para ingresar en una BBDD'''

    diccionario_datos_productos={"ID":id,"NAME":name,"INFO":info,"LISTA_URL":lista_URLs,"REGULAR_PRICE":regular_prices,"DISCOUNT_PRICE":new_price}

    diccionario_comentarios_productos={"ID":id,"COMENTARIOS":comentarios}

    '''Listas para generar la información de los comentarios'''
    id_comment=[]
    comments=[]
    date=[]
    ratio=[]
    users=[]
    comment=[]

    print("Empezando a recoger datos de las paginas")
    for page in pages:
        
        if page == 1:
            URL = url
            response = requests.get(url)
            soup = bs(response.text, 'lxml')
            productos = soup.find_all(class_='caption')
            for producto in productos[9:]:
                URL_producto = producto.find('a')['href']
                lista_URLs.append(URL_producto)
            
        else:
            URL = url+'page' + str(page)+'/'
            response = requests.get(URL)
            soup = bs(response.text, 'lxml')
            productos = soup.find_all(class_='caption')
            for producto in productos[9:]:
                URL_producto = producto.find('a')['href']
                lista_URLs.append(URL_producto)
    print("Terminando de recoger los datos de los links de las paginas\nEmpezando a generar las listas de los productos")

    for i in range(len(lista_URLs)):
        id.append(i)

        
    '''Extraemos la información de cada producto existente'''

    for URL in lista_URLs:
        url_product=URL
        response_product = requests.get(url_product)
        soup_product = bs(response_product.text, 'lxml')
        user_comments_product=[]
        date_comments_product=[]
        comments_product=[]
        rating=[]

        titulos=soup_product.find_all("h1",class_="h3")
        for titulo in titulos:
            nombre=titulo.get_text(strip=True)
            name.append(nombre)

        all_price = soup_product.find_all("div", class_="productoPrecio pull-right tdd_precio")                        
        for price_container in all_price:                                                                    
            try:
                special_price = price_container.find("span", class_="productSpecialPrice")
                if special_price:
                    item_price = float(special_price.get_text(strip=True).replace(",", ".").split('€')[0])
                    new_price.append(item_price)
                    regular_price = price_container.find("del").get_text(strip=True)
                    item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                    regular_prices.append(item_regular_price)
                else:
                    regular_price = price_container.find("span").get_text(strip=True)
                    item_regular_price = float(regular_price.replace(",", ".").split('€')[0])
                    new_price.append(item_regular_price)
                    regular_prices.append(None)
            except:
                new_price.append(None)
                regular_prices.append(None)

        description=soup_product.find("div", class_="description") 
        information=description.get_text().split('\n')[1:]
        documentation = ''.join(information)
        info.append(documentation)


        '''Vamos a obtener los datos de los comentarios de los usuarios'''
        # print("Cargando los datos de los comentarios")

        all_user_comments = soup_product.find_all("span", class_="name-user") 
        for user_comment in all_user_comments:
            user_comments_product.append(user_comment.get_text(strip=True))

        
        all_dates = soup_product.find_all("span", class_="date")  
        for dates in all_dates:
            dates_text=dates.get_text(strip=True)
            # dates=datetime.strftime(dates, '%dd/%mm/%Y')
            date_comments_product.append(dates_text)
            # date_object = datetime.strptime(date_comments_product)

        all_comments = soup_product.find_all("p")
        for formats in all_comments[-len(date_comments_product):]:
            comments_product.append(formats.get_text(strip=True))

        hearts = soup_product.find_all('div', class_= 'box-description')
        for heart in hearts:
            heart_rating = heart.find_all('span', class_= 'fas fa-heart')
            num_hearts = len(heart_rating)
            rating.append(num_hearts)

        datos = list(zip(date_comments_product,rating, user_comments_product,comments_product ))
        comentarios.append(datos)

    for i, regular_price in enumerate(regular_prices):
        if regular_price is None:
            regular_prices[i] = new_price[i]

    print("Realizando la ingenieria de los datos\nEliminando duplicados de productos")
    productos=pd.DataFrame(diccionario_datos_productos)
    noduplicated_product = productos.drop_duplicates(subset='NAME', keep='first')
    removed_id = productos[productos.duplicated(subset='NAME', keep='first')]['ID']

    print("Tratando los comentarios")

    comentarios_productos=pd.DataFrame(diccionario_comentarios_productos)
    comentarios=pd.DataFrame()
    diccionario={"id":id_comment,"comments":comments}

    for id_product,n_comments in enumerate (comentarios_productos['COMENTARIOS']):
        for i in n_comments:
            id_comment.append(id_product)
            comments.append(i)


    for j in range(len(diccionario['comments'])):
        date.append(diccionario['comments'][j][0])
        ratio.append(diccionario['comments'][j][1])
        users.append(diccionario['comments'][j][2])
        comment.append(diccionario['comments'][j][3])


    comentarios['ID']=pd.Series(id_comment)
    comentarios['DATE']=pd.Series(date)
    comentarios['RATIO']=pd.Series(ratio)
    comentarios['USERS']=pd.Series(users)
    comentarios['COMMENT']=pd.Series(comment)
    
    noduplicated_comments = comentarios[~comentarios['ID'].isin(productos[productos['ID'].isin(removed_id)]['ID'])]
    
    h=input("Quieres salvar los datos?").upper()
    if h=="SI":
        noduplicated_product.to_csv(product_ingest,header=True,index=False)           # Tengo que generar el path correcto
        noduplicated_comments.to_csv(comment_ingest,header=True,index=True)           # Tengo que generar el path correcto
    
    return noduplicated_product,noduplicated_comments



In [ ]:
'''Generando nuevos dataframes'''
def product_engineer(df_product,product_engineer=product_engineer):
    """
    Realiza ingeniería de datos para la creación de la tabla de productos.

    Input:
    - df_product (pd.DataFrame): DataFrame de productos.

    Return:
    - df_product (pd.DataFrame): DataFrame de productos con ingeniería aplicada.
    """
    
    print("Generando el fichero de productos")
    
    # df_product=pd.read_csv(folder_ingest+file_ingest_product+'_'+date+ext)
    df_product['NAME'] = df_product['NAME'].str.replace(r'-(?=\w)', '_')
    df_product[['PRODUCT', 'SLOGAN']] = df_product['NAME'].str.split('[,-.]', 1, expand=True)
    df_product['PRODUCT'] = df_product['PRODUCT'].str.strip()
    df_product['SLOGAN'] = df_product['SLOGAN'].str.strip()
    df_product['CHARACTERISTICS'] = df_product['INFO'].str.split('Ver características y medidas|Características', 1).str[1]
    df_product['DESCRIPTION'] = df_product['INFO'].str.split('Ver características y medidas|Características', 1).str[0].str.strip()
    df_product['CHARACTERISTICS'] = df_product['CHARACTERISTICS'].str.replace('\r', ' ')
    df_product['DESCRIPTION'] = df_product['DESCRIPTION'].str.replace('\r', ' ')

    col_1 = df_product.pop('PRODUCT')
    col_2=df_product.pop('SLOGAN')
    col_3=df_product.pop('DESCRIPTION')
    col_4=df_product.pop('CHARACTERISTICS')

    df_product.drop(columns=['NAME'],inplace=True)
    df_product.drop(columns=['INFO'],inplace=True)

    df_product.insert(loc= 1 , column= 'PRODUCT', value= col_1)
    df_product.insert(loc= 2 , column= 'SLOGAN', value= col_2)
    df_product.insert(loc= 3 , column= 'DESCRIPTION', value= col_3)
    df_product.insert(loc= 4 , column= 'CHARACTERISTICS', value= col_4)
    df_product=df_product.iloc[:,:6]
    # df_product.to_csv(folder_ingest+file_product+'_'+date+ext,header=True,index=False)
    h=input("Quieres salvar los datos?").upper()
    if h=="SI":
        df_product.to_csv(product_engineer,header=True,index=False)           # Tengo que generar el path correcto
        
    return df_product

def tag_engineer(df_tag,nombre_listas=nombre_listas,tag_engineer=tag_engineer):
    """
    Realiza ingeniería de datos para la creación de la tabla de tags.

    Input:
    - df_tag (pd.DataFrame): DataFrame de productos.
    - nombre_lista (Lista): Lista de nombres utilizada para cargar listas desde pickles.

    Return:
    - df_tag (pd.DataFrame): DataFrame de tags de los diversos productos.
    """
    from Utils.functions import eliminacion_acentos,cargar_listas_desde_pickles,aplicar_funcion_a_columna
    print("Generando el fichero de tags")
    # df_tag=pd.read_csv(folder_ingest+file_product+'_'+date+ext)
    df_tag['SLOGAN'] = df_tag['SLOGAN'].str.lower()
    df_tag['DESCRIPTION'] = df_tag['DESCRIPTION'].str.lower()
    # df_tag['SLOGAN'] = df_tag['SLOGAN'].apply(eliminacion_acentos)              # Esto da error, debe de ser porque hay NaN en el campo
    df_tag['DESCRIPTION'] = df_tag['DESCRIPTION'].apply(eliminacion_acentos)
    listas = cargar_listas_desde_pickles(nombre_listas)
    for nombre_lista in listas:
        df_tag = aplicar_funcion_a_columna(df_tag, listas,nombre_lista)
    h=input("Quieres salvar los datos?").upper()
    if h=="SI":
        df_tag.to_csv(tag_engineer,header=True,index=False)           # Tengo que generar el path correcto

    return df_tag

def price_engineer (dataframe,price_engineer=price_engineer):
    """
    Realiza ingeniería de datos para la creación de la tabla de precios.

    Input:
    - dataframe (pd.DataFrame): DataFrame de productos.

    Return:
    - df_prices (pd.DataFrame): DataFrame de precios de los productos.
    """

    print("Generando el fichero de precios")
    # dataframe=pd.read_csv(folder_ingest+file_ingest_product+'_'+date+ext)
    col_1 = dataframe.pop('REGULAR_PRICE')
    col_2=dataframe.pop('DISCOUNT_PRICE')
    col_3=dataframe['ID']
    dataframe.insert(loc= 1 , column= 'ID_PRODUCT', value= col_3)
    dataframe.insert(loc= 2 , column= 'REGULAR_PRICE', value= col_1)
    dataframe.insert(loc= 3 , column= 'DISCOUNT_PRICE', value= col_2)
    date_price=datetime.today().strftime('%y/%m/%d')
    df_prices=dataframe.iloc[:,:4]
    df_prices['FECHA']=date_price
    h=input("Quieres salvar los datos?").upper()
    if h=="SI":
        df_prices.to_csv(price_engineer,header=True,index=False)           # Tengo que generar el path correcto
    return df_prices

def comments_engineer (dataframe,dm_mapping=dm_mapping,comment_engineer=comment_engineer,user_engineer=user_engineer):
    """
    Realiza ingeniería de datos para la creación de la tabla de comentarios.

    Input:
    - dataframe (pd.DataFrame): DataFrame de comentarios.
    - dm_mapping (dict): Mapeo de meses para el tratamiento de fechas.

    Return:
    - df_comments (pd.DataFrame): DataFrame de comentarios con ingeniería aplicada.
    - df_users (pd.DataFrame): DataFrame de usuarios que han realizado comentarios.

    """

    '''Tratando a los Usuarios'''
    print("Generando el fichero de usuarios")

    nombre_count = {}
    count = {}

    for i, row in dataframe.iterrows():
        id = row['ID']
        nombre = row['USERS']
        
        if id not in count:
            count[id] = {}
            
        if nombre in count[id]:
            count[id][nombre] += 1
            nueva_nombre = f"{nombre}_{count[id][nombre]}"
            dataframe.loc[i, 'USERS'] = nueva_nombre
        else:
            count[id][nombre] = 1
    lista_users=dataframe['USERS'].unique()
    mivalor = [ x for x in range(len(lista_users))]             
    lista_users=list(lista_users)                                
    lista_users_code = {k: v for k, v in zip(lista_users, mivalor)}   
    dataframe['ID_USERS']= dataframe['USERS'].map(lista_users_code)
    df_users=pd.DataFrame()
    df_users['ID_USERS']=dataframe['ID_USERS']
    df_users['USERS']=dataframe['USERS']
    df_users.drop_duplicates(subset='ID_USERS', keep='first',inplace=True)



    print("Generando el fichero de comentarios")
    '''Tratando a las Fechas'''

    dataframe['DAY']=dataframe['DATE'].str.split(' ').str.get(1).astype('Int64')
    dataframe['MONTH']=dataframe['DATE'].str.split(' ').str.get(2).str.split(',').str.get(0)
    dataframe['YEAR']=dataframe['DATE'].str.split(' ').str.get(-1).astype('Int64')
    dataframe['MONTH']=dataframe['MONTH'].map(dm_mapping)
    dataframe['DATE'] = pd.to_datetime(dataframe.iloc[:,-3:])
    df_comments=dataframe.iloc[:,:-3]

    col = df_comments.pop('ID_USERS')
    df_comments.drop(columns=['USERS'],inplace=True)
    df_comments.insert(loc= 4 , column= 'ID_USERS', value= col)
    h=input("Quieres salvar los datos?").upper()
    if h=="SI":
        df_users.to_csv(user_engineer,header=True,index=False)           # Tengo que generar el path correcto
        df_comments.to_csv(comment_engineer,header=True,index=False)           # Tengo que generar el path correcto

    return df_comments,df_users





Probando las funciones

In [ ]:
noduplicated_product, noduplicated_comments = spider_amantis(url_secundaria)
df_product = product_engineer(noduplicated_product)    
# df_product.head()
# df_tag=tag_engineer(df_product,nombre_listas)
# df_tag.head()


In [ ]:
df_product.head()

In [ ]:
def tag_engineer(df_tag,nombre_listas=nombre_listas,tag_engineer=tag_engineer):
    """
    Realiza ingeniería de datos para la creación de la tabla de tags.

    Input:
    - df_tag (pd.DataFrame): DataFrame de productos.
    - nombre_lista (Lista): Lista de nombres utilizada para cargar listas desde pickles.

    Return:
    - df_tag (pd.DataFrame): DataFrame de tags de los diversos productos.
    """

    print("Generando el fichero de tags")
    # df_tag=pd.read_csv(folder_ingest+file_product+'_'+date+ext)
    df_tag['SLOGAN'] = df_tag['SLOGAN'].str.lower()
    df_tag['DESCRIPTION'] = df_tag['DESCRIPTION'].str.lower()
    # df_tag['SLOGAN'] = df_tag['SLOGAN'].apply(f.eliminacion_acentos)              # Esto da error, debe de ser porque hay NaN en el campo
    df_tag['DESCRIPTION'] = df_tag['DESCRIPTION'].apply(f.eliminacion_acentos)
    listas = f.cargar_listas_desde_pickles(nombre_listas)
    # for nombre_lista in listas:
    #     df_tag = f.aplicar_funcion_a_columna(df_tag, listas,nombre_lista)
    # h=input("Quieres salvar los datos?").upper()
    # if h=="SI":
    #     df_tag.to_csv(tag_engineer,header=True,index=False)           # Tengo que generar el path correcto

    return df_tag,listas

In [ ]:
df_tag,listas=tag_engineer(df_product)
df_tag.head()
# ,nombre_listas=nombre_listas,tag_engineer=tag_engineer)

In [ ]:
# noduplicated_product, noduplicated_comments = spider_amantis(url_secundaria)
noduplicated_product.head()

**Hay que tener en cuenta que las funciones que dejamos finalmente en el fichero .py sufren modificaciones sobre estas.**

## Modificaciones oportunas para adaptar la información a cargar en la BBDDD.

En este sentido tenemos que entender que la BBDD ya tiene determinados productos cargados y, por lo tanto, con su ID establecidas.

Es por ello que hay que cambiar las ID de la descarga a las ID existentes.

¿Cómo lo vamos a realizar? Vamos a extraer la información de la BBDD en un Dataframe y compararlos para cambiar las ID de los campos *[PRODUCT.product]* y *[USERS.users]*.

También hay que tener en cuenta que la descarga hemos obtenido toda la información de *COMMENTS*, por lo que hay que ver cual es la fecha más reciente de un comentario en la BBDD y solo dejar en el dataframe de COMMENTS los comentarios no guardados. 




#### 1. Reduciendo los comentarios.

¿Qué librerías necesitaremos? 

    - Las librerías para llamar a la BBDD.
    - Las librerías para el manejo de Dataframes.
    - Las librerías para el manejo de Fechas.

In [ ]:
import os
os.chdir(os.path.split(os.getcwd())[0])
folder=os.getcwd()

In [15]:
folder

'd:\\Data_science\\Javier\\Repositorios\\Proyecto_tienda_online\\src'

In [14]:
import pandas as pd
import numpy as np
from datetime import datetime,date
import sqlite3
from Utils import functions as f

In [16]:
df_date=pd.read_csv(r'Data\comentarios_240301.csv')
df_date.head()

,ID,DATE,RATIO,COMMENT,ID_USERS
0,0,2024-02-29,5,"Es una pasada, es pequeñito pero tiene mucha p...",0
1,0,2023-10-21,5,muy práctico por su pequeño tamaño. vibración ...,1
2,0,2023-10-01,5,"Es pequeño pero útil, da mucho juego y la pila...",2
3,0,2023-09-12,4,"Magnifica bala, es muy potente y da mucho jueg...",3
4,0,2023-08-22,5,Pequeño y potente. Se puede combinar con otros...,4


In [18]:
df_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3609 entries, 0 to 3608
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   ID        3609 non-null   int64 
 1   DATE      3609 non-null   object
 2   RATIO     3609 non-null   int64 
 3   COMMENT   3609 non-null   object
 4   ID_USERS  3609 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 141.1+ KB


In [26]:
df_date['DATE']

0       2024-02-29
1       2023-10-21
2       2023-10-01
3       2023-09-12
4       2023-08-22
           ...    
3604    2023-08-23
3605    2023-08-18
3606    2022-02-21
3607    2020-03-12
3608    2019-09-05
Name: DATE, Length: 3609, dtype: object

In [17]:
dm_mapping={
    'enero':1, 
    'febrero':2, 
    'marzo':3, 
    'abril':4, 
    'mayo':5,
    'junio':6, 
    'julio':7,
    'agosto':8, 
    'septiembre':9, 
    'octubre':10, 
    'noviembre':11, 
    'diciembre':12,
} 

In [41]:
def reengineer_comment(BBDD, df_comment):
    '''Reducción del número de registros del dataframe commentarios para su ingesta en la BBDD
    Input:
    - BBDD (str): Base de datos utilizada.
    - df_comment (Dataframe): Dataframe con los comentarios originales, sin filtrar

    Return:
    - df_comment_filtered (Dataframe): Dataframe con los comentarios filtrados y las fechas convertidas en Datetime.
    '''
    print("Iniciando la conversión de fecha y la reducción de datos de comentarios")

    ''' Conectamos con la base de datos, extraemos la fecha más reciente y la cerramos'''
    conn = sqlite3.connect(BBDD)
    cursor = conn.cursor()
    query='''SELECT MAX(DATE) FROM COMMENT'''
    MAX_date=f.sql_query(query,cursor)
    conn.commit()
    cursor.close()
    conn.close()
    fecha_maxima = MAX_date.iloc[0, 0]
    fecha_maxima = date.fromisoformat(fecha_maxima)
    max_date = pd.to_datetime(fecha_maxima, unit='ns')

    '''Cargamos el dataframe y lo preparamos para su filtro'''
    df_comment['DATE']=pd.to_datetime(df_comment['DATE'])
    df_comment_filtered=df_comment[df_comment['DATE']> max_date]
    print("El número de registros a ingresar es:",len(df_comment_filtered))
    return df_comment_filtered

df_date_filtered=reengineer_comment("Resources/online_shop.db",df_date)




Iniciando la conversión de fecha y la reducción de datos de comentarios
El número de registros a ingresar es: 383


#### 2. Reindexando ***PRODUCTOS*** u ***USUARIOS***.

¿Qué librerías necesitaremos? 

    - Las librerías para llamar a la BBDD.
    - Las librerías para el manejo de Dataframes.


In [2]:
import os
import pandas as pd
import sqlite3
os.chdir(os.path.split(os.getcwd())[0])
folder=os.getcwd()
from Utils import functions as f


In [19]:
df_product=pd.read_csv(r'Data\productos_240301.csv')
df_product.head()

,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL
0,0,TOK®,el Vibrador Sumergible Más Vendido de amantis,Descubre el encanto y la funcionalidad con TOK...,NaN,https://www.amantis.net/tok-el-vibrador-sumerg...
1,1,MUST,Vibrador Interactivo de Larga Distancia,MUST es un vibrador interactivo de última gene...,Destacadas:Control Remoto a Través de Aplicac...,https://www.amantis.net/must-vibrador-interact...
2,2,TOBOGANE,el vibrador doble más vendido,"Por favor, desabróchense los cinturones de seg...","Vibrador con doble motor. Medidas: 19cm, (11c...",https://www.amantis.net/tobogane-el-vibrador-d...
3,3,KUCHI,minisuccionador sónico de amantis,KUCHI ¡Pero qué monada de succionador de clíto...,Succionador sónicoMaterial: silicona médica y...,https://www.amantis.net/kuchi-minisuccionador-...
4,4,PACHA,Succionador y vibrador punto G,"Como casi todo el mundo sabe, el succionador a...",Vibrador con succión sónicaMedidas: 13 cm larg...,https://www.amantis.net/pacha-succionador-y-vi...


In [20]:
conn = sqlite3.connect("Resources/online_shop.db")
cursor = conn.cursor()
query='''SELECT ID,URL FROM PRODUCT'''
df_bbdd=f.sql_query(query,cursor)
conn.commit()
cursor.close()
conn.close()
df_bbdd.head()

,ID,URL
0,0,https://www.amantis.net/masaboom-el-gran-masaj...
1,1,https://www.amantis.net/massaji-potente-masaje...
2,2,https://www.amantis.net/digit-pro-dedal-vibrad...
3,3,https://www.amantis.net/mini-carnival-mini-mas...
4,4,https://www.amantis.net/lingus-vibrador-sexo-o...


In [4]:
df_bbdd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      561 non-null    int64 
 1   URL     561 non-null    object
dtypes: int64(1), object(1)
memory usage: 8.9+ KB


In [21]:
df_productos_coincidentes = pd.merge(df_bbdd, df_product, left_on="URL", right_on="LISTA_URL")
df_productos_coincidentes.head()

,ID_x,URL,ID_y,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL
0,11,https://www.amantis.net/tobogane-hot-rabbit-el...,33,TOBOGANE HOT RABBIT,el superventas de amantis ¡mejorado!,Vuelve nuestro vibrador de doble estimulación ...,"Medidas: 19 cm (11 cm insertables) y 3,3 cm/ 2...",https://www.amantis.net/tobogane-hot-rabbit-el...
1,13,https://www.amantis.net/meneo-sube-baja-realis...,31,MENEO sube y baja,placer realista con control remoto,"Si te gusta que te metan un buen meneo, hazte ...",Dildo percutor y vibrador con control remoto3 ...,https://www.amantis.net/meneo-sube-baja-realis...
2,15,https://www.amantis.net/tandem-2-flex-vibrador...,29,TANDEM 2 flex: La Evolución del Placer en Pareja,NaN,Innovación y Versatilidad en TANDEM 2 flex Te ...,Principales:Vibraciones Personalizables: 10 p...,https://www.amantis.net/tandem-2-flex-vibrador...
3,18,https://www.amantis.net/tobogane-el-vibrador-d...,2,TOBOGANE,el vibrador doble más vendido,"Por favor, desabróchense los cinturones de seg...","Vibrador con doble motor. Medidas: 19cm, (11c...",https://www.amantis.net/tobogane-el-vibrador-d...
4,20,https://www.amantis.net/lizo-2-dildo-suave-sil...,10,LIZO 2,El Dildo Versátil en Silicona Suave,"Explora la versatilidad sin igual de LIZO 2, t...",NaN,https://www.amantis.net/lizo-2-dildo-suave-sil...


In [ ]:
len(df_productos_coincidentes)

In [22]:
id_producto_BBDD = df_productos_coincidentes['ID_x'].tolist()
id_producto_actual = df_productos_coincidentes['ID_y'].tolist()

In [23]:
map_product = {k: v for k, v in zip(id_producto_actual,id_producto_BBDD)}
map_product

{33: 11,
 31: 13,
 29: 15,
 2: 18,
 10: 20,
 34: 22,
 94: 24,
 16: 26,
 12: 27,
 50: 28,
 104: 33,
 186: 35,
 36: 37,
 114: 41,
 39: 43,
 40: 44,
 19: 47,
 30: 57,
 15: 60,
 217: 61,
 58: 63,
 156: 66,
 103: 70,
 227: 73,
 42: 74,
 105: 82,
 100: 83,
 65: 92,
 20: 96,
 25: 108,
 120: 131,
 44: 134,
 90: 135,
 13: 136,
 96: 139,
 190: 150,
 89: 152,
 9: 155,
 7: 157,
 140: 160,
 71: 168,
 107: 170,
 121: 173,
 225: 180,
 123: 183,
 118: 188,
 75: 191,
 138: 198,
 112: 206,
 47: 213,
 98: 214,
 143: 222,
 175: 231,
 132: 233,
 79: 246,
 81: 251,
 60: 253,
 116: 262,
 117: 263,
 153: 268,
 180: 270,
 53: 273,
 169: 275,
 52: 286,
 212: 287,
 106: 289,
 113: 296,
 167: 298,
 49: 308,
 76: 309,
 174: 321,
 74: 325,
 181: 329,
 91: 339,
 54: 348,
 240: 352,
 162: 356,
 211: 357,
 141: 358,
 133: 360,
 48: 369,
 154: 372,
 109: 375,
 126: 391,
 242: 398,
 243: 404,
 200: 407,
 111: 416,
 46: 435,
 241: 436,
 239: 471,
 97: 473,
 176: 479,
 207: 500,
 127: 501,
 198: 504,
 78: 520,
 230: 532,


In [24]:
max_id_product=df_bbdd['ID'].max()
max_id_product

575

In [25]:
df_product_out = pd.merge(df_bbdd, df_product, left_on="URL", right_on="LISTA_URL",how='right',suffixes=('_',''))
df_product_out = df_product_out[df_product_out['URL'].isnull()]
df_product_out

,ID_,URL,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL
0,NaN,NaN,0,TOK®,el Vibrador Sumergible Más Vendido de amantis,Descubre el encanto y la funcionalidad con TOK...,NaN,https://www.amantis.net/tok-el-vibrador-sumerg...
1,NaN,NaN,1,MUST,Vibrador Interactivo de Larga Distancia,MUST es un vibrador interactivo de última gene...,Destacadas:Control Remoto a Través de Aplicac...,https://www.amantis.net/must-vibrador-interact...
3,NaN,NaN,3,KUCHI,minisuccionador sónico de amantis,KUCHI ¡Pero qué monada de succionador de clíto...,Succionador sónicoMaterial: silicona médica y...,https://www.amantis.net/kuchi-minisuccionador-...
4,NaN,NaN,4,PACHA,Succionador y vibrador punto G,"Como casi todo el mundo sabe, el succionador a...",Vibrador con succión sónicaMedidas: 13 cm larg...,https://www.amantis.net/pacha-succionador-y-vi...
6,NaN,NaN,6,SuctionMaster,Poderoso Desarrollador de Pene Automático,El SuctionMaster es un dispositivo revoluciona...,Destacadas:Bomba de Succión: Tecnología de va...,https://www.amantis.net/suctionmaster-desarrol...
...,...,...,...,...,...,...,...,...
187,NaN,NaN,232,BolasBen_Wametálicas,edición 50 Sombras de Grey,El rubor del movimiento constante de las Silve...,"• Suaves, frescas y sólidas para sensacione...",https://www.amantis.net/bolas-ben-wa-metalicas...
188,NaN,NaN,233,Tira de bolas anales de silicona Emotions,NaN,Emotions son las tiras anales de silicona que ...,· Contiene: 7 cuentas anales con anilla de ex...,https://www.amantis.net/tira-bolas-anales-sili...
189,NaN,NaN,234,EDGAR,"huevo vibrador ¡Vibran los dos, mando y huevo!","¡Vibran los dos, mando y huevo!Lo tenía que so...",100% silicona muy suave sin ftalatosMotor muy ...,https://www.amantis.net/edgar-huevo-vibrador-v...
190,NaN,NaN,235,Bolas anales de silicona,edición 50 Sombras de Grey,"Te quiero dolorida, nena. Quiero que, cada vez...",• Perfecta para principiantes en juegos ana...,https://www.amantis.net/bolas-anales-silicona-...


In [26]:
id_product_out = df_product_out['ID'].tolist()

id_product_out

[0,
 1,
 3,
 4,
 6,
 8,
 14,
 17,
 18,
 21,
 22,
 23,
 24,
 26,
 27,
 28,
 35,
 37,
 43,
 45,
 51,
 55,
 56,
 57,
 59,
 61,
 63,
 67,
 68,
 69,
 77,
 80,
 82,
 83,
 84,
 85,
 86,
 88,
 92,
 95,
 99,
 101,
 110,
 115,
 119,
 122,
 129,
 130,
 135,
 142,
 147,
 148,
 149,
 152,
 158,
 159,
 160,
 161,
 164,
 165,
 170,
 171,
 172,
 173,
 177,
 182,
 183,
 184,
 185,
 189,
 191,
 192,
 195,
 197,
 201,
 202,
 203,
 204,
 208,
 213,
 214,
 215,
 216,
 220,
 221,
 222,
 223,
 231,
 232,
 233,
 234,
 235,
 244]

In [27]:
df_product_out = df_product_out[df_product.columns]
df_product_out['ID'] = max_id_product + 1 + df_product_out.index
df_product_out

C:\Users\Javier\AppData\Local\Temp\ipykernel_15200\3943084143.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_product_out['ID'] = max_id_product + 1 + df_product_out.index


,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL
0,576,TOK®,el Vibrador Sumergible Más Vendido de amantis,Descubre el encanto y la funcionalidad con TOK...,NaN,https://www.amantis.net/tok-el-vibrador-sumerg...
1,577,MUST,Vibrador Interactivo de Larga Distancia,MUST es un vibrador interactivo de última gene...,Destacadas:Control Remoto a Través de Aplicac...,https://www.amantis.net/must-vibrador-interact...
3,579,KUCHI,minisuccionador sónico de amantis,KUCHI ¡Pero qué monada de succionador de clíto...,Succionador sónicoMaterial: silicona médica y...,https://www.amantis.net/kuchi-minisuccionador-...
4,580,PACHA,Succionador y vibrador punto G,"Como casi todo el mundo sabe, el succionador a...",Vibrador con succión sónicaMedidas: 13 cm larg...,https://www.amantis.net/pacha-succionador-y-vi...
6,582,SuctionMaster,Poderoso Desarrollador de Pene Automático,El SuctionMaster es un dispositivo revoluciona...,Destacadas:Bomba de Succión: Tecnología de va...,https://www.amantis.net/suctionmaster-desarrol...
...,...,...,...,...,...,...
187,763,BolasBen_Wametálicas,edición 50 Sombras de Grey,El rubor del movimiento constante de las Silve...,"• Suaves, frescas y sólidas para sensacione...",https://www.amantis.net/bolas-ben-wa-metalicas...
188,764,Tira de bolas anales de silicona Emotions,NaN,Emotions son las tiras anales de silicona que ...,· Contiene: 7 cuentas anales con anilla de ex...,https://www.amantis.net/tira-bolas-anales-sili...
189,765,EDGAR,"huevo vibrador ¡Vibran los dos, mando y huevo!","¡Vibran los dos, mando y huevo!Lo tenía que so...",100% silicona muy suave sin ftalatosMotor muy ...,https://www.amantis.net/edgar-huevo-vibrador-v...
190,766,Bolas anales de silicona,edición 50 Sombras de Grey,"Te quiero dolorida, nena. Quiero que, cada vez...",• Perfecta para principiantes en juegos ana...,https://www.amantis.net/bolas-anales-silicona-...


In [28]:
id_new_product = df_product_out['ID'].tolist()
map_product_out = {k: v for k, v in zip(id_product_out ,id_new_product)}

map_product_out

{0: 576,
 1: 577,
 3: 579,
 4: 580,
 6: 582,
 8: 584,
 14: 589,
 17: 592,
 18: 593,
 21: 596,
 22: 597,
 23: 598,
 24: 599,
 26: 601,
 27: 602,
 28: 603,
 35: 609,
 37: 611,
 43: 615,
 45: 617,
 51: 623,
 55: 627,
 56: 628,
 57: 629,
 59: 631,
 61: 633,
 63: 634,
 67: 637,
 68: 638,
 69: 639,
 77: 644,
 80: 647,
 82: 649,
 83: 650,
 84: 651,
 85: 652,
 86: 653,
 88: 654,
 92: 658,
 95: 660,
 99: 664,
 101: 666,
 110: 674,
 115: 679,
 119: 683,
 122: 686,
 129: 690,
 130: 691,
 135: 694,
 142: 698,
 147: 700,
 148: 701,
 149: 702,
 152: 703,
 158: 707,
 159: 708,
 160: 709,
 161: 710,
 164: 712,
 165: 713,
 170: 716,
 171: 717,
 172: 718,
 173: 719,
 177: 723,
 182: 726,
 183: 727,
 184: 728,
 185: 729,
 189: 732,
 191: 734,
 192: 735,
 195: 736,
 197: 737,
 201: 741,
 202: 742,
 203: 743,
 204: 744,
 208: 746,
 213: 749,
 214: 750,
 215: 751,
 216: 752,
 220: 754,
 221: 755,
 222: 756,
 223: 757,
 231: 762,
 232: 763,
 233: 764,
 234: 765,
 235: 766,
 244: 773}

Con esto tenemos 2 diccionarios.

Un diccionario con los indices de los productos coincidentes con la BBDD y un diccionario con los productos que no están en la BBDD.

Utilizaremos este último para filtrar y reducir el número de ingresos en las tablas *PRODUCTS* y *TAGS*

Utilizaremos la suma de los dos diccionarios para las tablas de *PRICES* y *COMMENTS*.

In [29]:
map_product.update(map_product_out)
len(map_product)

198

In [18]:
map_product

{11: 33,
 13: 31,
 15: 29,
 18: 2,
 20: 10,
 22: 34,
 24: 94,
 26: 16,
 27: 12,
 28: 50,
 33: 104,
 35: 186,
 37: 36,
 41: 114,
 43: 39,
 44: 40,
 47: 19,
 57: 30,
 60: 15,
 61: 217,
 63: 58,
 66: 156,
 70: 103,
 73: 227,
 74: 42,
 82: 105,
 83: 100,
 92: 65,
 96: 20,
 108: 25,
 131: 120,
 134: 44,
 135: 90,
 136: 13,
 139: 96,
 150: 190,
 152: 89,
 155: 9,
 157: 7,
 160: 140,
 168: 71,
 170: 107,
 173: 121,
 180: 225,
 183: 123,
 188: 118,
 191: 75,
 198: 138,
 206: 112,
 213: 47,
 214: 98,
 222: 143,
 231: 175,
 233: 132,
 246: 79,
 251: 81,
 253: 60,
 262: 116,
 263: 117,
 268: 153,
 270: 180,
 273: 53,
 275: 169,
 286: 52,
 287: 212,
 289: 106,
 296: 113,
 298: 167,
 308: 49,
 309: 76,
 321: 174,
 325: 74,
 329: 181,
 339: 91,
 348: 54,
 352: 240,
 356: 162,
 357: 211,
 358: 141,
 360: 133,
 369: 48,
 372: 154,
 375: 109,
 391: 126,
 398: 242,
 404: 243,
 407: 200,
 416: 111,
 435: 46,
 436: 241,
 471: 239,
 473: 97,
 479: 176,
 500: 207,
 501: 127,
 504: 198,
 520: 78,
 532: 230,


La función quedaría así:

In [ ]:
def mapeo_productos(BBDD, df_product):
   '''Función para la conversión de los ID de productos

   Input:
      - BBDD (str): nombre de la BBDD.

      - df_product (dataframe): Dataframe donde se encuentran los productos

   Output:
      - map_product (Dict): Diccionario con los ID mapeados coincidentes.
       
      - map_product_out (Dict): Diccionario con los ID mapeados no coincidentes.
   '''
   print("Vamos a reindexar los productos")


   '''Llamando a la base de datos para extraer información'''
   conn = sqlite3.connect(BBDD)
   cursor = conn.cursor()
   query='''SELECT ID,URL FROM PRODUCT'''
   df_bbdd=f.sql_query(query,cursor)
   conn.commit()
   cursor.close()
   conn.close()

   '''Obteniendo los ID coincidentes'''

   df_products_in = pd.merge(df_bbdd, df_product, left_on="URL", right_on="LISTA_URL")
   id_product_BBDD = df_products_in['ID_x'].tolist()
   id_product_new = df_products_in['ID_y'].tolist()
   map_product = {k: v for k, v in zip(id_product_BBDD, id_product_new)}

   '''Obteniendo los ID NO coincidentes'''
   max_id_product=df_bbdd['ID'].max()
   df_product_out = pd.merge(df_bbdd, df_product, left_on="URL", right_on="LISTA_URL",how='right',suffixes=('_',''))
   df_product_out = df_product_out[df_product_out['URL'].isnull()]
   id_product_out = df_product_out['ID'].tolist()
   df_product_out = df_product_out[df_product.columns]
   df_product_out['ID'] = max_id_product + 1 + df_product_out.index
   id_new_product = df_product_out['ID'].tolist()
   map_product_out = {k: v for k, v in zip(id_new_product,id_product_out )}
   map_product.update(map_product_out)

   return map_product, map_product_out


Vamos a realizar la re-indexación a continuación.

In [15]:
df_product.head()

,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL
0,0,TOK®,el Vibrador Sumergible Más Vendido de amantis,Descubre el encanto y la funcionalidad con TOK...,NaN,https://www.amantis.net/tok-el-vibrador-sumerg...
1,1,MUST,Vibrador Interactivo de Larga Distancia,MUST es un vibrador interactivo de última gene...,Destacadas:Control Remoto a Través de Aplicac...,https://www.amantis.net/must-vibrador-interact...
2,2,TOBOGANE,el vibrador doble más vendido,"Por favor, desabróchense los cinturones de seg...","Vibrador con doble motor. Medidas: 19cm, (11c...",https://www.amantis.net/tobogane-el-vibrador-d...
3,3,KUCHI,minisuccionador sónico de amantis,KUCHI ¡Pero qué monada de succionador de clíto...,Succionador sónicoMaterial: silicona médica y...,https://www.amantis.net/kuchi-minisuccionador-...
4,4,PACHA,Succionador y vibrador punto G,"Como casi todo el mundo sabe, el succionador a...",Vibrador con succión sónicaMedidas: 13 cm larg...,https://www.amantis.net/pacha-succionador-y-vi...


In [31]:
df_product.replace({'ID':map_product},inplace=True)
df_product.head()

,ID,PRODUCT,SLOGAN,DESCRIPTION,CHARACTERISTICS,LISTA_URL
0,576,TOK®,el Vibrador Sumergible Más Vendido de amantis,Descubre el encanto y la funcionalidad con TOK...,NaN,https://www.amantis.net/tok-el-vibrador-sumerg...
1,577,MUST,Vibrador Interactivo de Larga Distancia,MUST es un vibrador interactivo de última gene...,Destacadas:Control Remoto a Través de Aplicac...,https://www.amantis.net/must-vibrador-interact...
2,18,TOBOGANE,el vibrador doble más vendido,"Por favor, desabróchense los cinturones de seg...","Vibrador con doble motor. Medidas: 19cm, (11c...",https://www.amantis.net/tobogane-el-vibrador-d...
3,579,KUCHI,minisuccionador sónico de amantis,KUCHI ¡Pero qué monada de succionador de clíto...,Succionador sónicoMaterial: silicona médica y...,https://www.amantis.net/kuchi-minisuccionador-...
4,580,PACHA,Succionador y vibrador punto G,"Como casi todo el mundo sabe, el succionador a...",Vibrador con succión sónicaMedidas: 13 cm larg...,https://www.amantis.net/pacha-succionador-y-vi...


Procedemos igual con los ID de los USERS.

In [ ]:
def mapeo_usuarios(BBDD, df_user):
   '''Función para la conversión de los ID de productos

   Input:
      - BBDD (str): nombre de la BBDD.

      - df_user (dataframe): Dataframe donde se encuentran los productos

   Output:
      - map_user (Dict): Diccionario con los ID mapeados coincidentes.
       
      - map_user_out (Dict): Diccionario con los ID mapeados no coincidentes.
   '''
   print("Vamos a reindexar los productos")


   '''Llamando a la base de datos para extraer información'''
   conn = sqlite3.connect(BBDD)
   cursor = conn.cursor()
   query='''SELECT ID,USERS FROM USER'''
   df_bbdd=f.sql_query(query,cursor)
   conn.commit()
   cursor.close()
   conn.close()

   '''Obteniendo los ID coincidentes'''

   df_users_in = pd.merge(df_bbdd, df_user, left_on="USERS", right_on="USERS")
   id_user_BBDD = df_users_in['ID_x'].tolist()
   id_user_new = df_users_in['ID_y'].tolist()
   map_user = {k: v for k, v in zip(id_user_BBDD, id_user_new)}

   '''Obteniendo los ID NO coincidentes'''
   max_id_user=df_bbdd['ID'].max()
   df_users_out = pd.merge(df_bbdd, df_user, left_on="USERS", right_on="USERS",how='right',suffixes=('_',''))
   df_users_out = df_users_out[df_users_out['URL'].isnull()]
   id_user_out = df_users_out['ID'].tolist()
   df_users_out = df_users_out[df_user.columns]
   df_users_out['ID'] = max_id_user + 1 + df_users_out.index
   id_new_user = df_users_out['ID'].tolist()
   map_user_out = {k: v for k, v in zip(id_new_user,id_user_out )}
   map_user.update(map_user_out)

   return map_user, map_user_out

A partir de aqui procedemos al mapeado de los IDs correspondientes a cada dataframe y a la reducción del mismo en el caso de comments.

En cualquier caso, como hemos indicado antes en las tablas de *PRODUCTS*, *USERS*, *TAGS* solo se hará la ingesta de los datos nuevos, a partir de un **JOIN RIGHT**.

En el caso de *PRICES* se hará una ingesta total y en *COMMENTS* la ingesta indicada anteriormente (por fecha máxima de comentario en la BBDD).

In [11]:
BBDD="Resources/online_shop.db"
conn = sqlite3.connect(BBDD)
cursor = conn.cursor()
query_1='''SELECT ID,USERS FROM USERS'''
df_bbdd_user=f.sql_query(query_1,cursor)
query_2='''SELECT ID,URL FROM PRODUCT'''
df_bbdd_product=f.sql_query(query_2,cursor)
query_3='''SELECT * FROM TAGS'''
df_bbdd_tags=f.sql_query(query_3,cursor)

# conn.commit()
# cursor.close()
# conn.close()

In [12]:
df_bbdd_tags

,ID,JUGUETES,MUEBLES,LENCERIA,MASCULINO,FEMININO,ANAL,BDSM,AMENITIES
0,0,1,0,0,1,0,0,0,0
1,1,1,1,0,0,0,1,0,0
2,2,0,0,1,1,0,1,0,0
3,3,1,1,0,0,0,0,0,0
4,4,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
546,571,0,0,0,0,0,0,1,0
547,572,1,1,0,0,0,1,0,0
548,573,0,0,0,0,0,0,1,0
549,574,0,0,0,1,0,1,0,0


In [13]:
df_product=pd.read_csv(r'Data\productos_240301.csv')
df_tags=pd.read_csv(r'Data\tags_240301.csv')
df_users=pd.read_csv(r'Data\usuarios_240301.csv')
df_prices=pd.read_csv(r'Data\precios_240301.csv')
df_comment=pd.read_csv(r'Data\comentarios_240301.csv')


In [ ]:
'''Reducimos los datos de products y reindexamos'''
df_product_new = pd.merge(df_bbdd_product, df_product,left_on="URL", right_on="LISTA_URL",how='right',suffixes=('_',''))
df_product_new = df_product_new[df_product_new['URL'].isnull()]
df_product_new.replace({'ID':map_product},inplace=True)

'''Reducimos los datos de users y reindexamos'''
df_users_new = pd.merge(df_bbdd_user, df_users, left_on="USERS", right_on="USERS",how='right',suffixes=('_',''))
df_users_new = df_users_new[df_users_new['URL'].isnull()]
df_users_new.replace({'ID':map_user},inplace=True)

'''Reindexamos los datos de prices'''
df_prices_new=df_prices.replace({'ID':map_product},inplace=True)

'''Reducimos los datos de comments y reindexamos'''
df_comment_new=reengineer_comment("Resources/online_shop.db",df_comment)
df_comment_new.replace({'ID_PRODUCT':map_product},inplace=True)
df_comment_new.replace({'ID_USERS':map_user},inplace=True)

# '''Reducimos los datos de tags y reindexamos'''
# df_tags_new = pd.merge(df_bbdd_product, df_tags,left_on="URL", right_on="LISTA_URL",how='right',suffixes=('_',''))
# df_tags_new = df_tags_new[df_tags_new['ID'].isnull()]
# df_tags_new.replace({'ID':map_product},inplace=True)




Cargando los datos en la BBDD.

In [ ]:

lista_valores_product = df_product_new.values.tolist()
cursor.executemany("INSERT INTO PRODUCTS VALUES (?,?,?,?,?,?)", lista_valores_product)

lista_valores_users = df_users_new.values.tolist()
cursor.executemany("INSERT INTO USERS VALUES (?,?)", lista_valores_users)

lista_valores_prices = df_prices_new.values.tolist()
cursor.executemany("INSERT INTO PRICES VALUES (?,?,?,?,?)", lista_valores_prices)

lista_valores_comments = df_comment_new.values.tolist()
cursor.executemany("INSERT INTO COMMENTS VALUES (?,?,?,?,?,?)", lista_valores_comments)

# lista_valores_tags = df_tags_new.values.tolist()
# cursor.executemany("INSERT INTO TAGS VALUES (?,?,?,?,?,?,?,?,?)", lista_valores_tags)



conn.commit()
cursor.close()
conn.close()